# Understand the model code

The model code is contained in the `wals_ml_engine` directory. The code's 
high-level functionality is implemented by the following files:

`mltrain.sh`
 * Launches various types of ML Engine jobs. This shell
      script accepts arguments for the location of the dataset file, the
      delimiter used to separate values in the file, and whether the data
      file has a header line. It's a best practice to create a script that
      automatically configures and executes ML Engine jobs.

`task.py`
 * Parses the arguments for the ML Engine job and executes
      training.

`model.py`
 * Loads the dataset.
 * Creates two sparse matrices from the data, one for  training and one for
      testing. Executes WALS on the training sparse matrix of ratings.

`wals.py`
 * Creates the WALS model.
 * Executes the WALS algorithm.
 * Calculates the root-mean-square error [RMSE](https://wikipedia.org/wiki/Root-mean-square_deviation) for a set of row/column factors and a ratings matrix.

**Execute the following cell to import necessary python modules:**

In [1]:
import datetime
import numpy as np
import os
import pandas as pd
from scipy.sparse import coo_matrix
import sh
import tensorflow as tf

## The MovieLens data set

**Execute the following cell to download the MovieLens 100k data set:**

In [ ]:
!curl -O 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
!unzip ml-100k.zip
!mkdir -p ../data
!cp ml-100k/u.data ../data/

**Let's take a look at the MovieLens data.** It consists of rows of individual user ratings of movies.  Each row consists of four elements: a unique user id, a unique 'item id' corresponding to a movie, a rating from 1-5, and a timestamp of when the rating was made.

In [3]:
!head ../data/u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


This model ignores the timestamp.  We will only make use of the **user id**, **item** (or movie) **id**, and **rating**.

## How the model preprocesses data

The model code performs data preprocessing to create a sparse ratings matrix and prepare it for matrix factorization. 

**This involves the following steps:**

 * The model code loads rows of data from a delimited text file, in this case the MovieLens data. 

**Execute this cell to read the MovieLens data set into a Pandas dataframe:**

In [8]:
input_file = '../data/u.data'
headers = ['user_id', 'item_id', 'rating', 'timestamp']
header_row = None
ratings_df = pd.read_csv(input_file,
                         sep='\t',
                         names=headers,
                         header=header_row,
                         dtype={
                           'user_id': np.int32,
                           'item_id': np.int32,
                           'rating': np.float32,
                           'timestamp': np.int32,
                         })

 * The code establishes a 0-indexed set of unique IDs for users and items. This guarantees that a unique ID corresponds to specific row and column indexes of the sparse ratings matrix.
 
**Let's find the number of unique users and movies:**

In [18]:
np_users = ratings_df.user_id.as_matrix()
np_items = ratings_df.item_id.as_matrix()
unique_users = np.unique(np_users)
unique_items = np.unique(np_items)

n_users = unique_users.shape[0]
n_items = unique_items.shape[0]

In [22]:
n_users

943

In [23]:
n_items

1682

The MovieLens 100k data uses 1-based IDs where the lowest index of the unique set is 1. But the TensorFlow code assumes a starting index of zero.  The code handles this **by converting the Pandas dataframe to a numpy array and subtracting one from each index:**

In [9]:
ratings = ratings_df.as_matrix(['user_id', 'item_id', 'rating'])
ratings[:,0] -= 1
ratings[:,1] -= 1

In [10]:
ratings

array([[1.950e+02, 2.410e+02, 3.000e+00],
       [1.850e+02, 3.010e+02, 3.000e+00],
       [2.100e+01, 3.760e+02, 1.000e+00],
       ...,
       [2.750e+02, 1.089e+03, 1.000e+00],
       [1.200e+01, 2.240e+02, 2.000e+00],
       [1.100e+01, 2.020e+02, 3.000e+00]])

 * The model code randomly selects a test set of ratings. By default, 10% of the ratings are chosen for the test set. These ratings are removed from the training set and will be used to evaluate the predictive accuracy of the user and item factors.

In [11]:
TEST_SET_RATIO = 10

test_set_size = len(ratings) / TEST_SET_RATIO
test_set_idx = np.random.choice(xrange(len(ratings)),
                                size=test_set_size, replace=False)
test_set_idx = sorted(test_set_idx)

ts_ratings = ratings[test_set_idx]
tr_ratings = np.delete(ratings, test_set_idx, axis=0)

In [14]:
test_set_idx

[2,
 15,
 31,
 34,
 40,
 56,
 64,
 79,
 85,
 109,
 110,
 115,
 122,
 131,
 132,
 141,
 155,
 157,
 172,
 181,
 187,
 209,
 220,
 228,
 229,
 233,
 250,
 258,
 269,
 276,
 282,
 285,
 292,
 309,
 315,
 319,
 322,
 326,
 345,
 353,
 357,
 360,
 364,
 368,
 369,
 390,
 402,
 404,
 409,
 429,
 440,
 443,
 504,
 527,
 528,
 536,
 540,
 542,
 561,
 562,
 578,
 586,
 589,
 591,
 602,
 607,
 616,
 621,
 634,
 637,
 641,
 642,
 645,
 647,
 648,
 655,
 660,
 663,
 677,
 687,
 692,
 696,
 698,
 703,
 710,
 713,
 724,
 732,
 743,
 772,
 773,
 781,
 785,
 789,
 799,
 811,
 812,
 830,
 851,
 852,
 853,
 928,
 934,
 937,
 944,
 945,
 982,
 988,
 989,
 1022,
 1028,
 1032,
 1057,
 1060,
 1062,
 1066,
 1072,
 1081,
 1121,
 1132,
 1149,
 1156,
 1163,
 1168,
 1169,
 1208,
 1225,
 1228,
 1239,
 1246,
 1252,
 1273,
 1289,
 1312,
 1320,
 1324,
 1332,
 1335,
 1356,
 1373,
 1384,
 1385,
 1387,
 1394,
 1397,
 1423,
 1427,
 1436,
 1444,
 1471,
 1489,
 1502,
 1516,
 1521,
 1525,
 1578,
 1615,
 1630,
 1635,
 1648,

In [12]:
ts_ratings

array([[ 21., 376.,   1.],
       [302., 784.,   3.],
       [300.,  97.,   4.],
       ...,
       [278.,  63.,   1.],
       [912., 689.,   3.],
       [715., 203.,   5.]])

In [13]:
tr_ratings

array([[1.950e+02, 2.410e+02, 3.000e+00],
       [1.850e+02, 3.010e+02, 3.000e+00],
       [2.430e+02, 5.000e+01, 2.000e+00],
       ...,
       [2.750e+02, 1.089e+03, 1.000e+00],
       [1.200e+01, 2.240e+02, 2.000e+00],
       [1.100e+01, 2.020e+02, 3.000e+00]])

 * Finally, the code creates a `scipy` sparse matrix in coordinate form (`coo_matrix`) that includes the user and item indexes and ratings. The `coo_matrix` object acts as a wrapper for a sparse matrix. It also performs validation of the user and ratings indexes, checking for errors in preprocessing:

In [19]:
u_tr, i_tr, r_tr = zip(*tr_ratings)
tr_sparse = coo_matrix((r_tr, (u_tr, i_tr)), shape=(n_users, n_items))

u_ts, i_ts, r_ts = zip(*ts_ratings)
test_sparse = coo_matrix((r_ts, (u_ts, i_ts)), shape=(n_users, n_items))

In [20]:
tr_sparse

<943x1682 sparse matrix of type '<type 'numpy.float64'>'
	with 90000 stored elements in COOrdinate format>

In [21]:
test_sparse

<943x1682 sparse matrix of type '<type 'numpy.float64'>'
	with 10000 stored elements in COOrdinate format>

## How the WALS Algorithm is implemented in TensorFlow

You can read the [solution online](https://wikipedia.org/wiki/Root-mean-square_deviation) to understand how the WALS algorithm is implemented in TensorFlow.